In [241]:
file0 = """################################
#########.######################
#########..#####################
#########..G####################
########....#GG#################
########G......#################
########........################
###.####...#....################
#....###.###...G.###############
##......####.....#.#G..#.#######
###G.G...###.........#...#######
###......##...........##########
#............G#####...##########
#..G##G......#######..##########
#.G.#.......#########..#########
####..G.....#########...#.######
#...........#########..........#
##.#.....#..#########.E.....E..#
##.###..G.G.#########..........#
##...........#######E.#.......##
#.............#####..........###
#....#.....E................####
##.............##.E...........##
#....G.G.................###..##
#..............#.....E...###..##
#..##.##.G.....##E.......###.###
###G..##.......###.###...##...##
#####.E##.E.G..######...E.#..###
####...###..#..#######.......###
####...###############.#########
#####..#########################
################################"""

In [242]:
file1 = """#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########"""

r = [list(i) for i in file1.split('\n')]
r

[['#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', 'G', '.', '.', 'G', '.', '.', 'G', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'G', '.', '.', 'E', '.', '.', 'G', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'G', '.', '.', 'G', '.', '.', 'G', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [394]:
class Node():
    def __init__(self,x,y, parent = None):
        self.x = x
        self.y = y
        self.parent = parent
        self.childs = []
        self.leaf = False
        self.length = 0
        
        
        
    def __repr__(self):
        return f'({self.x},{self.y})'
    
    def add_child(self,child):
        self.childs.append(child)
        
    def get_range(self,rd):
        rango = []
        
        if rd[self.y+1][self.x] == '.':
            rango.append((self.x,self.y+1))
            
        if rd[self.y-1][self.x] == '.':
            rango.append((self.x,self.y-1))
            
        if rd[self.y][self.x-1] == '.':
            rango.append((self.x-1,self.y))
            
        if rd[self.y][self.x+1] == '.':
            rango.append((self.x+1,self.y))
            
        return rango
    
    def make_length(self):
        if self.parent is None:
            return self.length
        self.length+=1
        return self.parent.make_length()
    
    def get_length(self):
        self.length = 0
        self.make_length()
        return self.length
    
    def get_x(self):
        if self.parent.parent.parent is None:
            return self.parent.x
        return self.parent.get_x()
    
    def get_y(self):
        if self.parent.parent.parent is None:
            return self.parent.y
        return self.parent.get_y()
    
    
class Path():
    def __init__(self,master_node,rd, goals):
        self.master_node = master_node
        self.rd = rd
        self.current_level = 0
        self.nodes = {self.current_level:[master_node]}
        self.leafs = []
        self.posiciones= [(master_node.x, master_node.y)]
        self.goals = goals
        self.fin = False
        
    def add_node(self,node):
        self.nodes.append(node)
        
    def next_level(self):
        to_add = []
        
        self.nodes[self.current_level+1] = []
        
        for node in self.nodes[self.current_level]:
            for p in node.get_range(self.rd):
                if p not in self.posiciones:
                    
                    child = Node(p[0],p[1],node)
                    node.add_child(child)
                    if self.current_level == 0:
                        self.nodes[self.current_level+1].append(child)
                    to_add.append(p)
                    if p in self.goals:
                        child.leaf = True
                        self.leafs.append(child)
        ###
        if self.current_level !=0:
            child_list = []                
            for node in self.nodes[self.current_level]:
                for child in node.childs:
                    child_list.append(child)

            #sort child_list
            child_list.sort(key = lambda c:
                           (c.get_y(),c.get_x()))

            for child in child_list:
                if (child.x,child.y) not in [(n.x,n.y) for n in 
                                   self.nodes[self.current_level+1]]:
                    self.nodes[self.current_level+1].append(child)



            ###                
        if len(self.nodes[self.current_level+1])==0:
            self.fin = True
        if len(to_add) == 0:
            self.fin = True
            
        self.posiciones += to_add
        self.current_level += 1
        
    def fill(self):
        while (len(self.leafs) == 0) and (self.fin == False ):
            self.next_level()
        return self.next_move()
    
    def next_move(self):
        if self.fin == True:
            return None

        ne =  sorted([c for 
                       c in self.leafs],
              key = lambda n: (n.get_length(),
                               n.get_y(),n.get_x()))[0]
        return ne.get_x(),ne.get_y()
            

class Unit():
    def __init__(self,  x, y, tipo,ap):
        self.attack_power = ap
        self.hp = 200
        self.tipo = tipo 
        self.x = x
        self.y = y
        self.alive = True
        
    def attack(self,other_unit):
        other_unit.hp-=self.attack_power
        if other_unit.hp <=0:
            other_unit.alive = False
            #print('murio')
            #print(other_unit.alive)
            
    def get_range(self,rd):
        if not self.alive:
            return []
        rango = []
        if rd[self.y][self.x+1] == '.':
            rango.append((self.x+1, self.y))
        if rd[self.y][self.x-1] == '.':
            rango.append((self.x-1, self.y))
        if rd[self.y+1][self.x] == '.':
            rango.append((self.x, self.y+1))
        if rd[self.y-1][self.x] == '.':
            rango.append((self.x, self.y-1))
            
        return rango
    
    
    def __repr__(self):
        return f'[{self.tipo},{self.hp}]({self.x},{self.y})'
    

        

In [419]:
class Game():
    def __init__(self,rd,ap):
        
        self.rd = [list(i) for i in rd.split('\n')]
        self.ap = ap
        
    def build_units(self):
        self.units = []
        for n,fila in enumerate(self.rd):
            for m,columna in enumerate(fila):
                if columna == 'G':
                    self.units.append(Unit(m,n,'G',3))
                if columna == 'E':
                    self.units.append(Unit(m,n,'E',self.ap))
        self.units.sort(key = lambda u: (u.y,u.x))
        
    def get_enemies(self,unit):
        return sorted([u for u in self.units 
                       if u.alive and u.tipo != unit.tipo],
                     key = lambda p: (p.hp,p.y,p.x))
    
    def get_enemies_in_range(self,unit):
        enemies_in_range = []
        for enemy in self.get_enemies(unit):
            if (enemy.x,enemy.y) == (unit.x+1,unit.y):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x-1,unit.y):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x,unit.y+1):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x,unit.y-1):
                enemies_in_range.append(enemy)
        
        return sorted( enemies_in_range,
                     key = lambda p: (p.hp,p.y,p.x))
    
    def get_goals(self,unit):
        goals = []
        for u in [k for k in self.units if k.alive ]:
            if u.tipo != unit.tipo:
                goals += u.get_range(self.rd)
        return goals
    
                
                
                    
    
    def pass_turn(self):
        for u in self.units:
            #print('current_unit:' ,u)
            if not u.alive:
                continue
                
            
            #possible scenarios:
            # a) has a unit to attack -> attack
            # has no goals: continue
            # has no available movements: continue
            # can move: move
            
            #a) has a unit to attack:
            if len(self.get_enemies_in_range(u))>0:
                enemy = self.get_enemies_in_range(u)[0]
                #print(u,' attacks ',enemy)
                u.attack(enemy)
                if not enemy.alive:
                    self.rd[enemy.y][enemy.x] = '.'
                continue
            
            if len(self.get_goals(u)) == 0:
                continue
                
                
            if len(u.get_range(self.rd))==0:
                continue
            
            #if goals are in range, no need to fill path:
            
            if len([
                p for p in u.get_range(self.rd) if p in self.get_goals(u)
            ])>0:
                #sort goals that are in range
                new_x,new_y = sorted([g for g in self.get_goals(u) 
                        if g in u.get_range(self.rd)],
                       key = lambda m: (m[1],m[0])
                      )[0]
                #print(u,'->')
                self.rd[u.y][u.x] = '.'
                u.x,u.y = new_x,new_y
                self.rd[u.y][u.x] = u.tipo
                #print(u)
                
                if len(self.get_enemies_in_range(u))>0:
                    enemy = self.get_enemies_in_range(u)[0]
                    #print(u,' attacks ',enemy)
                    u.attack(enemy)
                    if not enemy.alive:
                        self.rd[enemy.y][enemy.x] = '.'
                    
                continue
            
            if len(u.get_range(self.rd))>0:
                
                    
                pati = Path(Node(u.x,u.y),self.rd,self.get_goals(u))

                try:
                    #if (u.x,u.y) == (10,19):
                        #print (self.get_goals(u))
                    new_x,new_y = pati.fill()
                    #if (u.x,u.y) == (10,19):
                        #print(new_x,new_y)
                        #print(pati.leafs)
                        #print(pati.nodes)
                        #print(self.units)
                        
                        #for i in self.rd:
                         #   print(''.join(i))
                        
                except TypeError:
                    #print(u,'failed try')
                    
                    continue

                #print(u,'->')

                self.rd[u.y][u.x] = '.'
                u.x,u.y = new_x,new_y
                self.rd[u.y][u.x] = u.tipo
                #print(u)
                
            if len(self.get_enemies_in_range(u))>0:
                enemy = self.get_enemies_in_range(u)[0]
                #print(u,' attacks ',enemy)
                u.attack(enemy)
                #print(u,' attacks ',enemy)
                if not enemy.alive:
                    self.rd[enemy.y][enemy.x] = '.'
                
            
            
        
    
    def play(self):
        
                
        self.escenario = {}
        turno = 0
        self.build_units()
        inicial = len([u for u in self.units if u.tipo == 'E'])
        while (len([u for u in self.units if u.tipo == 'G'])>0) and\
        (len([u for u in self.units if u.tipo == 'E'])>0):

            print(turno)
            print('goblins: ', [g.hp for g in self.units if g.tipo != 'E'])
            print('elfs: ', [g.hp for g in self.units if g.tipo != 'G'])
            for i in self.rd:
                print(''.join(i))
                
            #for k in self.units:
             #   print(k)

            
            
            self.pass_turn()
            turno+=1
            
            
                

            
            self.units = sorted([u for u in self.units if u.alive],
                            key = lambda p: (p.y,p.x))
            #for i in mg.rd:
             #   print(''.join(i))
        print('')
        final = len([u for u in self.units if u.tipo == 'E'])
        return inicial-final
                
            
            
                
        
        
            

In [379]:
mg = Game(file1,5)
mg.build_units()
mg.units
mg.play()

0,#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########
1,#########
#.G...G.#
#...G...#
#...E..G#
#.G.....#
#.......#
#G..G..G#
#.......#
#########
2,#########
#..G.G..#
#...G...#
#.G.E.G.#
#.......#
#G..G..G#
#.......#
#.......#
#########
3,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
4,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
5,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
6,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
7,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
8,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
9,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########
10,#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#......

1

In [378]:
mg.units

[[G,200](3,2),
 [G,128](4,2),
 [G,200](5,2),
 [G,200](3,3),
 [G,200](5,3),
 [G,200](1,4),
 [G,200](4,4),
 [G,200](7,5)]

In [259]:
for i in mg.rd:
    print(''.join(i))

#########
#.......#
#..GGG..#
#..G.G..#
#G..G...#
#......G#
#.......#
#.......#
#########


In [387]:
file_exp = """#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######"""

#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######

In [388]:
mg = Game(file_exp,4)
mg.build_units()
mg.play()

0,#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######
1,#######
#.EGEG#
#.#..E#
#E.##E#
#G.E#.#
#...#.#
#######
2,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
3,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
4,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
5,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
6,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
7,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
8,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
9,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
10,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
11,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
12,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
13,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
14,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
15,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
16,#######
#.EGEG#
#.#..E#
#E.##E#
#GE.#.#
#...#.#
#######
17,####

0

In [389]:
mg.units

[[E,200](2,1),
 [E,23](4,1),
 [E,200](3,2),
 [E,125](1,3),
 [E,200](5,3),
 [E,200](2,4)]

In [384]:
mg.units

[[G,196](1,1),
 [E,200](5,1),
 [E,194](1,2),
 [E,200](3,1),
 [E,200](5,2),
 [G,200](1,3),
 [E,200](5,4),
 [E,200](3,5)]

In [173]:
200+197+185+200+200

982

In [145]:
(200+197+185+200+200)*37

36334

In [178]:
file_exp2 = """#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######"""

In [262]:
mg = Game(file_exp2)
mg.build_units()
mg.play()

turno:  0  goblins hp:  800  elfs hp:  400
[E,200](4,2)  attacks  [G,200](5,2)
[G,197](5,2)  attacks  [E,200](4,2)
[G,200](5,3)  attacks  [E,200](5,4)
[E,197](5,4)  attacks  [G,200](5,3)
1, #######
#..G..#
#...EG#
#.#G#G#
#...#E#
#.....#
#######
turno:  1  goblins hp:  794  elfs hp:  394
[G,200](4,1)  attacks  [E,197](4,2)
[E,194](4,2)  attacks  [G,197](5,2)
[G,194](5,2)  attacks  [E,194](4,2)
[G,200](3,2)  attacks  [E,191](4,2)
[G,197](5,3)  attacks  [E,197](5,4)
[E,194](5,4)  attacks  [G,197](5,3)
2, #######
#...G.#
#..GEG#
#.#.#G#
#...#E#
#.....#
#######
turno:  2  goblins hp:  788  elfs hp:  382
[G,200](4,1)  attacks  [E,188](4,2)
[G,200](3,2)  attacks  [E,185](4,2)
[E,182](4,2)  attacks  [G,194](5,2)
[G,191](5,2)  attacks  [E,182](4,2)
[G,194](5,3)  attacks  [E,194](5,4)
[E,191](5,4)  attacks  [G,194](5,3)
3, #######
#...G.#
#..GEG#
#.#.#G#
#...#E#
#.....#
#######
turno:  3  goblins hp:  782  elfs hp:  370
[G,200](4,1)  attacks  [E,179](4,2)
[G,200](3,2)  attacks  [E,176](4,2)
[E,

In [125]:
mg.units

[[G,200](1,1), [G,131](2,2), [G,59](5,3), [G,200](5,5)]

In [126]:
200+131+59+200

590

In [237]:
file_3_exp = """#######
#.E...#
#.#..G#
#.###.#
#E#G#G#
#...#G#
#######"""

In [238]:
mg = Game(file_3_exp)
mg.build_units()
mg.play()

turno:  0  goblins hp:  800  elfs hp:  400
1, #######
#..E.G#
#.#...#
#.###G#
#.#.#.#
#E.G#G#
#######
turno:  1  goblins hp:  800  elfs hp:  400
[E,200](4,1)  attacks  [G,200](5,1)
[G,197](5,1)  attacks  [E,200](4,1)
[E,200](2,5)  attacks  [G,200](3,5)
[G,197](3,5)  attacks  [E,200](2,5)
2, #######
#...EG#
#.#..G#
#.###.#
#.#.#.#
#.EG#G#
#######
turno:  2  goblins hp:  794  elfs hp:  394
[E,197](4,1)  attacks  [G,197](5,1)
[G,194](5,1)  attacks  [E,197](4,1)
[G,200](4,2)  attacks  [E,194](4,1)
[E,197](2,5)  attacks  [G,197](3,5)
[G,194](3,5)  attacks  [E,197](2,5)
3, #######
#...EG#
#.#.G.#
#.###.#
#.#.#.#
#.EG#G#
#######
turno:  3  goblins hp:  788  elfs hp:  385
[E,191](4,1)  attacks  [G,194](5,1)
[G,191](5,1)  attacks  [E,191](4,1)
[G,200](4,2)  attacks  [E,188](4,1)
[E,194](2,5)  attacks  [G,194](3,5)
[G,191](3,5)  attacks  [E,194](2,5)
4, #######
#...EG#
#.#.G.#
#.###.#
#.#.#.#
#.EG#G#
#######
turno:  4  goblins hp:  782  elfs hp:  376
[E,185](4,1)  attacks  [G,191](5,1)
[G,188](5

In [129]:
mg.units

[[G,200](3,2), [G,98](1,5), [G,38](3,5), [G,200](5,5)]

In [130]:
200+98+38+200

536

In [183]:
536*54

28944

In [184]:
536*55

29480

In [337]:
mg = Game(file0)
mg.build_units()
mg.play()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,

In [339]:
mg.units

[[G,2](21,12),
 [G,200](8,18),
 [G,200](11,18),
 [G,200](12,19),
 [G,200](12,20),
 [G,200](20,20),
 [G,200](9,21),
 [G,200](11,21),
 [G,80](13,21),
 [G,98](19,21),
 [G,200](9,22),
 [G,200](12,22),
 [G,2](5,23),
 [G,200](10,23),
 [G,59](12,23),
 [G,125](15,23),
 [G,200](9,24)]

In [340]:
sum([k.hp for k in mg.units])

2566

In [342]:
2566*105

269430

In [263]:
file_5_exp = """#######
#E.G#.#
#.#G..#
#G.#.G#
#G..#.#
#...E.#
#######"""

In [264]:
mg = Game(file_5_exp)
mg.build_units()
mg.play()

turno:  0  goblins hp:  1000  elfs hp:  400
[E,200](2,1)  attacks  [G,200](3,1)
[G,197](3,1)  attacks  [E,200](2,1)
[E,200](5,5)  attacks  [G,200](5,4)
1, #######
#.EG#.#
#G#G..#
#..#..#
#.G.#G#
#....E#
#######
turno:  1  goblins hp:  994  elfs hp:  397
[E,197](2,1)  attacks  [G,197](3,1)
[G,194](3,1)  attacks  [E,197](2,1)
[G,200](1,1)  attacks  [E,194](2,1)
[G,197](5,4)  attacks  [E,200](5,5)
[E,197](5,5)  attacks  [G,197](5,4)
2, #######
#GEG#.#
#.#G..#
#..#..#
#..G#G#
#....E#
#######
turno:  2  goblins hp:  988  elfs hp:  388
[G,200](1,1)  attacks  [E,191](2,1)
[E,188](2,1)  attacks  [G,194](3,1)
[G,191](3,1)  attacks  [E,188](2,1)
[G,194](5,4)  attacks  [E,197](5,5)
[E,194](5,5)  attacks  [G,194](5,4)
3, #######
#GEG#.#
#.#G..#
#..#..#
#...#G#
#..G.E#
#######
turno:  3  goblins hp:  982  elfs hp:  379
[G,200](1,1)  attacks  [E,185](2,1)
[E,182](2,1)  attacks  [G,191](3,1)
[G,188](3,1)  attacks  [E,182](2,1)
[G,191](5,4)  attacks  [E,194](5,5)
[G,200](4,5)  attacks  [E,191](5,5)
[E

In [267]:
f10="""################################
#################.....##########
#################..#.###########
#################.........######
##################......########
#################G.GG###########
###############...#..###########
###############......G..########
############..G.........########
##########.G.....G......########
##########......#.........#..###
##########...................###
#########G..G.#####....E.G.E..##
######..G....#######...........#
#######.....#########.........##
#######..#..#########.....#.####
##########..#########..G.##..###
###########G#########...E...E.##
#########.G.#########..........#
#########GG..#######.......##.E#
######.G......#####...##########
#...##..G..............#########
#...#...........###..E.#########
#.G.............###...##########
#................###############
##.........E.....###############
###.#..............#############
###..G........E.....############
###......E..........############
###......#....#E#...############
###....####.#...##.#############
################################"""

In [268]:
mg = Game(f10)
mg.build_units()
mg.play()

turno:  0  goblins hp:  4000  elfs hp:  2000
[E,200](24,12)  attacks  [G,200](25,12)
[G,197](25,12)  attacks  [E,200](24,12)
[E,200](26,12)  attacks  [G,197](25,12)
[G,200](24,16)  attacks  [E,200](24,17)
[E,197](24,17)  attacks  [G,200](24,16)
1, turno:  1  goblins hp:  3991  elfs hp:  1994
[E,197](24,12)  attacks  [G,194](25,12)
[G,191](25,12)  attacks  [E,197](24,12)
[E,200](26,12)  attacks  [G,191](25,12)
[G,197](24,16)  attacks  [E,197](24,17)
[E,194](24,17)  attacks  [G,197](24,16)
[E,200](8,27)  attacks  [G,200](7,27)
2, turno:  2  goblins hp:  3979  elfs hp:  1988
[E,194](24,12)  attacks  [G,188](25,12)
[G,185](25,12)  attacks  [E,194](24,12)
[E,200](26,12)  attacks  [G,185](25,12)
[G,194](24,16)  attacks  [E,194](24,17)
[E,191](24,17)  attacks  [G,194](24,16)
[E,200](11,22)  attacks  [G,200](11,21)
[G,197](7,27)  attacks  [E,200](8,27)
[E,197](8,27)  attacks  [G,197](7,27)
3, turno:  3  goblins hp:  3964  elfs hp:  1979
[E,191](24,12)  attacks  [G,182](25,12)
[G,179](25,12)  a

[G,200](24,11)  attacks  [E,50](24,12)
[G,200](26,11)  attacks  [E,185](26,12)
[G,200](23,12)  attacks  [E,47](24,12)
[E,44](24,12)  attacks  [G,47](25,12)
[G,44](25,12)  attacks  [E,44](24,12)
[E,182](26,12)  attacks  [G,44](25,12)
[G,179](22,13)  attacks  [E,179](22,14)
[G,200](24,13)  attacks  [E,41](24,12)
[E,200](25,13)  attacks  [G,41](25,12)
[G,200](21,14)  attacks  [E,176](22,14)
[E,173](22,14)  attacks  [G,179](22,13)
[E,200](23,16)  attacks  [G,107](24,16)
[G,104](24,16)  attacks  [E,143](24,17)
[E,140](24,17)  attacks  [G,104](24,16)
[G,149](11,21)  attacks  [E,29](11,22)
[G,200](10,22)  attacks  [E,26](11,22)
[E,23](11,22)  attacks  [G,149](11,21)
[G,200](12,22)  attacks  [E,23](11,22)
[G,161](11,23)  attacks  [E,20](11,22)
[E,161](12,23)  attacks  [G,161](11,23)
[G,200](13,23)  attacks  [E,161](12,23)
[G,200](12,24)  attacks  [E,158](12,23)
[G,200](8,26)  attacks  [E,116](8,27)
[G,113](7,27)  attacks  [E,113](8,27)
[E,110](8,27)  attacks  [G,113](7,27)
[E,200](7,28)  attac

[E,200](23,16)  attacks  [G,35](24,16)
[G,32](24,16)  attacks  [E,107](24,17)
[E,104](24,17)  attacks  [G,32](24,16)
[G,200](12,22)  attacks  [E,29](12,23)
[G,125](11,23)  attacks  [E,26](12,23)
[E,23](12,23)  attacks  [G,125](11,23)
[G,200](13,23)  attacks  [E,23](12,23)
[G,200](12,24)  attacks  [E,20](12,23)
[G,200](8,26)  attacks  [E,17](8,27)
[G,41](7,27)  attacks  [E,14](8,27)
[E,11](8,27)  attacks  [G,41](7,27)
[G,200](9,27)  attacks  [E,11](8,27)
[E,200](7,28)  attacks  [G,38](7,27)
[G,200](8,28)  attacks  [E,8](8,27)
32, turno:  32  goblins hp:  3227  elfs hp:  901
[G,179](26,11)  attacks  [E,125](26,12)
[G,200](25,12)  attacks  [E,122](26,12)
[E,119](26,12)  attacks  [G,179](26,11)
[G,140](22,13)  attacks  [E,77](22,14)
[G,179](24,13)  attacks  [E,173](25,13)
[E,170](25,13)  attacks  [G,179](24,13)
[G,200](21,14)  attacks  [E,74](22,14)
[E,71](22,14)  attacks  [G,140](22,13)
[G,200](23,14)  attacks  [E,71](22,14)
[E,200](23,16)  attacks  [G,29](24,16)
[G,26](24,16)  attacks  [

[E,89](7,28)  attacks  [G,143](6,28)
[G,200](8,28)  attacks  [E,89](7,28)
[G,200](7,29)  attacks  [E,86](7,28)
45, turno:  45  goblins hp:  3013  elfs hp:  526
[G,140](26,11)  attacks  [E,47](26,12)
[G,200](25,12)  attacks  [E,44](26,12)
[E,41](26,12)  attacks  [G,140](26,11)
[G,122](24,13)  attacks  [E,59](24,14)
[E,149](25,13)  attacks  [G,122](24,13)
[G,185](23,14)  attacks  [E,56](24,14)
[E,53](24,14)  attacks  [G,119](24,13)
[G,107](22,15)  attacks  [E,188](23,15)
[E,185](23,15)  attacks  [G,107](22,15)
[G,200](23,16)  attacks  [E,185](23,15)
[G,140](6,28)  attacks  [E,83](7,28)
[E,80](7,28)  attacks  [G,140](6,28)
[G,200](8,28)  attacks  [E,80](7,28)
[G,200](7,29)  attacks  [E,77](7,28)
46, turno:  46  goblins hp:  2998  elfs hp:  499
[G,137](26,11)  attacks  [E,41](26,12)
[G,200](25,12)  attacks  [E,38](26,12)
[E,35](26,12)  attacks  [G,137](26,11)
[G,116](24,13)  attacks  [E,53](24,14)
[E,149](25,13)  attacks  [G,116](24,13)
[G,185](23,14)  attacks  [E,50](24,14)
[E,47](24,14) 

56, ################################
#################.....##########
#################..#.###########
#################.........######
##################......########
#################....###########
###############...#..###########
###############.........########
############............########
##########..............########
##########......#.........#..###
##########........G..........###
#########.....#####......G....##
######.......#######....GEG....#
#######.....#########..G......##
#######..#..#########.GEG.#.####
##########..#########..G.##..###
###########.#########.........##
#########...#########..........#
#########...G#######.......##..#
######..G...G.#####...##########
#...##...G.......G.....#########
#...#...........###....#########
#...............###...##########
#.....GG.........###############
##.....G.........###############
###.#.G............#############
###.................############
###.................############
###......#....#.#...############
###...

63, ################################
#################.....##########
#################..#.###########
#################.........######
##################......########
#################....###########
###############...#..###########
###############.........########
############............########
##########..............########
##########......#.........#..###
##########...............G...###
#########.....#####......G....##
######.....G.#######....GEG....#
#######.....#########..G......##
#######..#..#########.GEG.#.####
##########..#########..G.##..###
###########.#########.........##
#########..G#########G.........#
#########....#######.......##..#
######.GG....G#####...##########
#...##......G..........#########
#...#....G......###....#########
#.......G.......###...##########
#................###############
##...............###############
###.#..............#############
###.................############
###.................############
###......#....#.#...############
###...

In [270]:
2812*69

194028

In [274]:
f11 = """################################
#######.G...####################
#########...####################
#########.G.####################
#########.######################
#########.######################
#########G######################
#########.#...##################
#########.....#..###############
########...G....###.....########
#######............G....########
#######G....G.....G....#########
######..G.....#####..G...#######
######...G...#######......######
#####.......#########....G..E###
#####.####..#########G...#....##
####..####..#########..G....E..#
#####.####G.#########...E...E.##
#########.E.#########.........##
#####........#######.E........##
######........#####...##...#..##
###...................####.##.##
###.............#########..#####
#G#.#.....E.....#########..#####
#...#...#......##########.######
#.G............#########.E#E####
#..............##########...####
##..#..........##########.E#####
#..#G..G......###########.######
#.G.#..........#################
#...#..#.......#################
################################"""

In [332]:
mg = Game(f11)
mg.build_units()
mg.play()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,

In [334]:
sum([k.hp for k in mg.units])

2419

In [336]:
for k in mg.rd:
    print(''.join(k))

################################
#######.....####################
#########...####################
#########...####################
#########.######################
#########.######################
#########.######################
#########.#...##################
#########.....#..###############
########......G.###.....########
#######.........G.......########
#######...GG..G........#########
######........#####......#######
######.......#######......######
#####.......#########.G.G....###
#####.####..#########..G.#....##
####..####..#########...G......#
#####.####..#########...G.....##
#########...#########GGG......##
#####........#######..........##
######........#####...##...#..##
###...........G.......####.##.##
###.............#########..#####
#.#.#...........#########..#####
#...#...#......##########.######
#..............#########..#.####
#..............##########...####
##..#..........##########..#####
#..#..........###########.######
#...#..........#################
#...#..#..

In [289]:
ff = """################################
#######..G..####################
#########...####################
#########...####################
#########.######################
#########.######################
#########G######################
#########.#...##################
#########....G#..###############
########........###.....########
#######.................########
#######...GGGG.........#########
######........#####......#######
######.......#######......######
#####.......#########.....GE.###
#####.####..#########....#E...##
####..####..#########G..GE.....#
#####.####GG#########G.GE.....##
#########.GE#########EGE......##
#####.....GG.#######.GE.......##
######........#####...##...#..##
###...........G.......####.##.##
###.............#########..#####
#.#.#...........#########..#####
#...#...#......##########.######
#..............#########.E#.####
#..............##########...####
##..#..........##########..#####
#..#..........###########.######
#...#..........#################
#...#..#.......#################
################################"""

In [292]:
mg= Game(ff)
mg.build_units()

In [294]:
for n,l in enumerate(mg.rd):
    print(n,''.join(l))

0 ################################
1 #######..G..####################
2 #########...####################
3 #########...####################
4 #########.######################
5 #########.######################
6 #########G######################
7 #########.#...##################
8 #########....G#..###############
9 ########........###.....########
10 #######.................########
11 #######...GGGG.........#########
12 ######........#####......#######
13 ######.......#######......######
14 #####.......#########.....GE.###
15 #####.####..#########....#E...##
16 ####..####..#########G..GE.....#
17 #####.####GG#########G.GE.....##
18 #########.GE#########EGE......##
19 #####.....GG.#######.GE.......##
20 ######........#####...##...#..##
21 ###...........G.......####.##.##
22 ###.............#########..#####
23 #.#.#...........#########..#####
24 #...#...#......##########.######
25 #..............#########.E#.####
26 #..............##########...####
27 ##..#..........##########..#####
28

In [296]:
mg.units[23]

[G,200](10,19)

In [303]:
mg.get_goals(mg.units[23])

[(28, 14),
 (27, 15),
 (27, 15),
 (26, 16),
 (26, 16),
 (25, 17),
 (25, 17),
 (24, 18),
 (24, 18),
 (23, 19),
 (23, 19),
 (24, 25),
 (25, 26),
 (25, 24)]

In [304]:
mg.units[23].get_range(mg.rd)

[(9, 19), (10, 20)]

In [301]:
pati = Path(Node(mg.units[23].x,mg.units[23].y),mg.rd,mg.get_goals(mg.units[23]))

In [302]:
x,y = pati.fill()

TypeError: cannot unpack non-iterable NoneType object

In [300]:
pati.leafs

[]

In [420]:
mg = Game(file0,19)
mg.build_units()
mg.play()

0
goblins:  [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
elfs:  [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
################################
#########.######################
#########..#####################
#########..G####################
########....#GG#################
########G......#################
########........################
###.####...#....################
#....###.###...G.###############
##......####.....#.#G..#.#######
###G.G...###.........#...#######
###......##...........##########
#............G#####...##########
#..G##G......#######..##########
#.G.#.......#########..#########
####..G.....#########...#.######
#...........#########..........#
##.#.....#..#########.E.....E..#
##.###..G.G.#########..........#
##...........#######E.#.......##
#.............#####..........###
#....#.....E................####
##.............##.E...........##
#....G.G.................###..##
#..............#.....E...###..##
#..#

9
goblins:  [200, 200, 200, 200, 200, 200, 105, 200, 200, 200, 200, 200, 48, 200, 200, 143, 29, 48, 200, 200]
elfs:  [185, 200, 200, 152, 200, 200, 200, 200, 149, 176]
################################
#########.######################
#########..#####################
#########...####################
########....#..#################
########.....G.#################
########........################
###.####...#....################
#....###.###..G..###############
##......####G....#.#...#.#######
###......###.GG......#...#######
###......##...........##########
#.....G.......#####...##########
#...##.......#######.G##########
#...#.G.....#########E.#########
####...G....#########...#.######
#......G....#########..E.......#
##.#.....#..#########..........#
##.###..G.G.#########..........#
##.........G.#######E.#.......##
#.........GE..#####..........###
#....#...G.GE....E..........####
##.........E...##.............##
#....................E...###..##
#..............#.........###..##
#..##.#

24
goblins:  [200, 200, 124, 200, 200, 181, 10, 200, 200, 67, 143]
elfs:  [155, 200, 200, 164, 59, 176, 200, 152, 200, 59]
################################
#########.######################
#########..#####################
#########...####################
########....#..#################
########.......#################
########........################
###.####...#....################
#....###.###.....###############
##......####G....#.#...#.#######
###......###.....G.GE#...#######
###......##........G.E##########
#.........G...#####...##########
#...##.......#######..##########
#...#.......#########..#########
####........#########...#.######
#...........#########..........#
##.#.....#..#########..........#
##.###......#########..........#
##........GGE#######..#.......##
#........GEE..#####..........###
#....#..G.E.................####
##......GE.....##.............##
#.......E................###..##
#..............#.........###..##
#..##.##...E...##........###.###
###...##..EG...###.

39
goblins:  [67, 10]
elfs:  [83, 200, 200, 134, 140, 164, 200, 188, 17, 50]
################################
#########.######################
#########..#####################
#########...####################
########....#..#################
########.......#################
########........################
###.####...#....################
#....###.###.....###############
##......####.....#.#...#.#######
###......###........E#...#######
###......##........E..##########
#.............#####...##########
#...##.......#######..##########
#...#.......#########..#########
####....E...#########...#.######
#.......E.E.#########..........#
##.#...E.#GG#########..........#
##.###.E..EE#########..........#
##....E......#######..#.......##
#.............#####..........###
#....#......................####
##.............##.............##
#........................###..##
#..............#.........###..##
#..##.##.......##........###.###
###...##.......###.###...##...##
#####..##......######.....#..###

0

In [414]:
mg.units

[[E,83](18,10),
 [E,200](17,11),
 [E,134](8,14),
 [E,164](7,15),
 [E,200](10,15),
 [E,200](7,16),
 [E,134](10,16),
 [E,50](6,17),
 [E,188](10,18),
 [E,14](11,18)]

In [415]:
sum([k.hp for k in mg.units])

1367

In [410]:
1367*41

56047

In [411]:
1367*40

54680

In [412]:
1370*40

54800

In [416]:
mg = Game(file0,19)
mg.build_units()
mg.units

[[G,200](11,3),
 [G,200](13,4),
 [G,200](14,4),
 [G,200](8,5),
 [G,200](15,8),
 [G,200](20,9),
 [G,200](3,10),
 [G,200](5,10),
 [G,200](13,12),
 [G,200](3,13),
 [G,200](6,13),
 [G,200](2,14),
 [G,200](6,15),
 [E,200](22,17),
 [E,200](28,17),
 [G,200](8,18),
 [G,200](10,18),
 [E,200](20,19),
 [E,200](11,21),
 [E,200](18,22),
 [G,200](5,23),
 [G,200](7,23),
 [E,200](21,24),
 [G,200](9,25),
 [E,200](17,25),
 [G,200](3,26),
 [E,200](6,27),
 [E,200](10,27),
 [G,200](12,27),
 [E,200](24,27)]

In [421]:
sum([83, 200, 134, 200, 164, 137, 200, 50, 188, 14])

1370